In [1]:
import torch

# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
# check if cuda is available
print(torch.cuda.is_available())

True
True
False


/Users/rylanspence/opt/anaconda3/envs/diffusion/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import h5py
import numpy as np
import time
import os

# import filters
import Estkf,Ensrf,Senkf
import Estkf_torch,Ensrf_torch,Senkf_torch



In [3]:
def stats(x):
    return x.shape, np.mean(x,axis=1), np.cov(x[:1000,:],ddof=1)

def close(state_analysis,xb):
    return np.allclose(state_analysis.mean(axis=1),xb.mean(axis=1))

def timeit(filter,filter_object):
    """
    function execution time
    """
    st = time.time()
    x = filter_object._assimilate()
    et = time.time()

    # get the execution time
    elapsed_time = et - st
    
    return f"Filter: {filter.__name__} Execution time: {elapsed_time} seconds"

In [4]:
hf = h5py.File('../data/example_sim_outputs.h5',"r")
zeta = np.array(hf.get('zeta'))
obs = np.array(hf.get('obs'))
    
print(f"Ensemble state matrix dimensions (ne x nx): {zeta.shape}")
print(f"Observation vector at one time step dimensions: (ny) {obs.shape}")


state_forecast = zeta.T[:,:5]                            # ensemble of state vectors
observations = obs                                      # model observations (adcirc output file fort.63.nc)                  
model_observations = np.eye(obs.size)[:,:5]             # observation operator ( adcirc output file fort.61.nc)

params = [state_forecast,model_observations,observations]

Ensemble state matrix dimensions (ne x nx): (10, 3352598)
Observation vector at one time step dimensions: (ny) (11,)


In [8]:
filters = [Estkf,Ensrf,Senkf,Estkf_torch,Ensrf_torch,Senkf_torch]

for i, filter in enumerate(filters):
    filter_class= getattr(filter,filter.__name__.lower())
    filter_object= filter_class(*params)
    print(timeit(filter,filter_object))
   

Filter: Estkf Execution time: 0.25817012786865234 seconds
Filter: Ensrf Execution time: 0.3384850025177002 seconds
Filter: Senkf Execution time: 0.06730914115905762 seconds
Filter: Estkf_torch Execution time: 0.15881013870239258 seconds
Filter: Ensrf_torch Execution time: 0.08200287818908691 seconds
Filter: Senkf_torch Execution time: 0.07541608810424805 seconds


In [7]:
# print(stats(x1),"\n")
# print(stats(x2),"\n")
# print(close(x1,x2),"\n")
